In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": "fa2073f0-882e-4087-baec-64e8e7d3d33a",
          "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope="kradls",key="ADLSAppKey"),
          "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/a3000236-0406-45f9-b95f-d7f32bc39886/oauth2/token"}

# Optionally, you can add <directory-name> to the source URI of your mount point.
dbutils.fs.mount(
  source = "abfss://development@teststorage19923.dfs.core.windows.net/",
  mount_point = "/mnt/Sales",
  extra_configs = configs)

Out[3]: True

In [0]:
dbutils.fs.ls("/mnt/Sales/sap/data-files/")

Out[10]: [FileInfo(path='dbfs:/mnt/Sales/sap/data-files/Personal_data_sap.txt', name='Personal_data_sap.txt', size=256)]

In [0]:
df_pers = spark.read.csv("/mnt/Sales/sap/data-files/Personal_data_sap.txt", sep='\t',header=True,inferSchema=True)

In [0]:
df_pers.show(5)

+----------+---------------+-----------+----------------+----------+
P0000-PENR|TEXT-P0000-PENR|P0001-GESCH|TEXT-P0001-GESCH| P0002-DOB|
+----------+---------------+-----------+----------------+----------+
 103830| Rey Luke| M| Male|21.09.2000|
 103831| Lolly Leades| F| Female|01.09.2003|
 103832| Jordan K| M| Male|05.10.1985|
 103833| April Lapwood| F| Female|11.12.1995|
 103834| Joye Silson| M| Male|01.01.1998|
+----------+---------------+-----------+----------------+----------+

In [0]:
df_pers.printSchema()

root
-- P0000-PENR: integer (nullable = true)
-- TEXT-P0000-PENR: string (nullable = true)
-- P0001-GESCH: string (nullable = true)
-- TEXT-P0001-GESCH: string (nullable = true)
-- P0002-DOB: string (nullable = true)

In [0]:
#ADD SERIAL NUMBER
from pyspark.sql import Window

#import pyspark.sql.functions as F

from pyspark.sql import functions as F

window = Window.orderBy(F.col('P0000-PENR'))

df_pers = df_pers.withColumn('SerialNumber', F.row_number().over(window))

df_pers.show(5)

+----------+---------------+-----------+----------------+----------+------------+
P0000-PENR|TEXT-P0000-PENR|P0001-GESCH|TEXT-P0001-GESCH| P0002-DOB|SerialNumber|
+----------+---------------+-----------+----------------+----------+------------+
 103830| Rey Luke| M| Male|21.09.2000| 1|
 103831| Lolly Leades| F| Female|01.09.2003| 2|
 103832| Jordan K| M| Male|05.10.1985| 3|
 103833| April Lapwood| F| Female|11.12.1995| 4|
 103834| Joye Silson| M| Male|01.01.1998| 5|
+----------+---------------+-----------+----------------+----------+------------+

In [0]:
#RENAME COLUMNS
from functools import reduce

oldColumns = df_pers.columns[:-1]

newColumns = ["PersNo", "EmployeeName", "GenderKey", "Gender", "DOB"]

df_pers = reduce(lambda data, idx: data.withColumnRenamed(oldColumns[idx], newColumns[idx]), range(len(oldColumns)), df_pers)
df_pers.printSchema()
df_pers.show()

root
-- PersNo: integer (nullable = true)
-- EmployeeName: string (nullable = true)
-- GenderKey: string (nullable = true)
-- Gender: string (nullable = true)
-- DOB: string (nullable = true)
-- SerialNumber: integer (nullable = true)

+------+-------------+---------+------+----------+------------+
PersNo| EmployeeName|GenderKey|Gender| DOB|SerialNumber|
+------+-------------+---------+------+----------+------------+
103830| Rey Luke| M| Male|21.09.2000| 1|
103831| Lolly Leades| F|Female|01.09.2003| 2|
103832| Jordan K| M| Male|05.10.1985| 3|
103833|April Lapwood| F|Female|11.12.1995| 4|
103834| Joye Silson| M| Male|01.01.1998| 5|
+------+-------------+---------+------+----------+------------+

In [0]:
#ADD NECESSARY COLUMNS

df_pers = df_pers.withColumn("StartDate",F.lit(F.current_timestamp())).withColumn("EndDate",F.to_timestamp(F.lit("31.12.9999"), "dd.MM.yyyy")).withColumn("CreatedDate",F.lit(F.current_timestamp()))
df_pers.show()

+------+-------------+---------+------+----------+------------+--------------------+-------------------+--------------------+
PersNo| EmployeeName|GenderKey|Gender| DOB|SerialNumber| StartDate| EndDate| CreatedDate|
+------+-------------+---------+------+----------+------------+--------------------+-------------------+--------------------+
103830| Rey Luke| M| Male|21.09.2000| 1|2021-11-20 07:21:...|9999-12-31 00:00:00|2021-11-20 07:21:...|
103831| Lolly Leades| F|Female|01.09.2003| 2|2021-11-20 07:21:...|9999-12-31 00:00:00|2021-11-20 07:21:...|
103832| Jordan K| M| Male|05.10.1985| 3|2021-11-20 07:21:...|9999-12-31 00:00:00|2021-11-20 07:21:...|
103833|April Lapwood| F|Female|11.12.1995| 4|2021-11-20 07:21:...|9999-12-31 00:00:00|2021-11-20 07:21:...|
103834| Joye Silson| M| Male|01.01.1998| 5|2021-11-20 07:21:...|9999-12-31 00:00:00|2021-11-20 07:21:...|
+------+-------------+---------+------+----------+------------+--------------------+-------------------+--------------------+

In [0]:
#ADD ISActive Column
df_pers = df_pers.withColumn("IsActive",F.when((df_pers.EndDate >= F.current_timestamp()) & (df_pers.StartDate <= F.current_timestamp()), F.lit(True))
                  .otherwise(F.lit(False))
                  )
df_pers.show()

+------+-------------+---------+------+----------+------------+--------------------+-------------------+--------------------+--------+
PersNo| EmployeeName|GenderKey|Gender| DOB|SerialNumber| StartDate| EndDate| CreatedDate|IsActive|
+------+-------------+---------+------+----------+------------+--------------------+-------------------+--------------------+--------+
103830| Rey Luke| M| Male|21.09.2000| 1|2021-11-20 07:21:...|9999-12-31 00:00:00|2021-11-20 07:21:...| true|
103831| Lolly Leades| F|Female|01.09.2003| 2|2021-11-20 07:21:...|9999-12-31 00:00:00|2021-11-20 07:21:...| true|
103832| Jordan K| M| Male|05.10.1985| 3|2021-11-20 07:21:...|9999-12-31 00:00:00|2021-11-20 07:21:...| true|
103833|April Lapwood| F|Female|11.12.1995| 4|2021-11-20 07:21:...|9999-12-31 00:00:00|2021-11-20 07:21:...| true|
103834| Joye Silson| M| Male|01.01.1998| 5|2021-11-20 07:21:...|9999-12-31 00:00:00|2021-11-20 07:21:...| true|
+------+-------------+---------+------+----------+------------+--------------------+-------------------+--------------------+--------+

In [0]:
#Convert DataType of DOB

df_pers = df_pers.withColumn("DOB", F.to_timestamp(df_pers.DOB, "dd.MM.yyyy"))
df_pers.show()
df_pers.printSchema()

+------+-------------+---------+------+-------------------+------------+--------------------+-------------------+--------------------+--------+
PersNo| EmployeeName|GenderKey|Gender| DOB|SerialNumber| StartDate| EndDate| CreatedDate|IsActive|
+------+-------------+---------+------+-------------------+------------+--------------------+-------------------+--------------------+--------+
103830| Rey Luke| M| Male|2000-09-21 00:00:00| 1|2021-11-20 07:21:...|9999-12-31 00:00:00|2021-11-20 07:21:...| true|
103831| Lolly Leades| F|Female|2003-09-01 00:00:00| 2|2021-11-20 07:21:...|9999-12-31 00:00:00|2021-11-20 07:21:...| true|
103832| Jordan K| M| Male|1985-10-05 00:00:00| 3|2021-11-20 07:21:...|9999-12-31 00:00:00|2021-11-20 07:21:...| true|
103833|April Lapwood| F|Female|1995-12-11 00:00:00| 4|2021-11-20 07:21:...|9999-12-31 00:00:00|2021-11-20 07:21:...| true|
103834| Joye Silson| M| Male|1998-01-01 00:00:00| 5|2021-11-20 07:21:...|9999-12-31 00:00:00|2021-11-20 07:21:...| true|
+------+-------------+---------+------+-------------------+------------+--------------------+-------------------+--------------------+--------+

root
-- PersNo: integer (nullable = true)
-- EmployeeName: string (nullable = true)
-- GenderKey: string (nullable = true)
-- Gender: string (nullable = true)
-- DOB: timestamp (nullable = true)
-- SerialNumber: integer (nullable = true)
-- StartDate: timestamp (nullable = false)
-- EndDate: timestamp (nullable = true)
-- CreatedDate: timestamp (nullable = false)
-- IsActive: boolean (nullable = false)

In [0]:
#Add BatchID
import random
def generate_batch_id():
    return random.randint(0, 1000)

df_pers = df_pers.withColumn("BatchID", F.lit(generate_batch_id()))
df_pers.show()

+------+-------------+---------+------+-------------------+------------+--------------------+-------------------+--------------------+--------+-------+
PersNo| EmployeeName|GenderKey|Gender| DOB|SerialNumber| StartDate| EndDate| CreatedDate|IsActive|BatchID|
+------+-------------+---------+------+-------------------+------------+--------------------+-------------------+--------------------+--------+-------+
103830| Rey Luke| M| Male|2000-09-21 00:00:00| 1|2021-11-20 07:21:...|9999-12-31 00:00:00|2021-11-20 07:21:...| true| 199|
103831| Lolly Leades| F|Female|2003-09-01 00:00:00| 2|2021-11-20 07:21:...|9999-12-31 00:00:00|2021-11-20 07:21:...| true| 199|
103832| Jordan K| M| Male|1985-10-05 00:00:00| 3|2021-11-20 07:21:...|9999-12-31 00:00:00|2021-11-20 07:21:...| true| 199|
103833|April Lapwood| F|Female|1995-12-11 00:00:00| 4|2021-11-20 07:21:...|9999-12-31 00:00:00|2021-11-20 07:21:...| true| 199|
103834| Joye Silson| M| Male|1998-01-01 00:00:00| 5|2021-11-20 07:21:...|9999-12-31 00:00:00|2021-11-20 07:21:...| true| 199|
+------+-------------+---------+------+-------------------+------------+--------------------+-------------------+--------------------+--------+-------+

In [0]:
#ADD HASH COLUMNS
cols_with_allTS = df_pers.columns
cols_with_SD = [x for x in cols_with_allTS if x != "EndDate"]

#df_pers.withColumn("HashAllTS", F.sha2(F.concat_ws("\t", *cols), 256)).show()

df_pers = df_pers.withColumn("HashAllTS", F.md5(F.concat_ws("\t", *cols_with_allTS))).withColumn("HashWithStartDate", F.md5(F.concat_ws("\t", *cols_with_SD)))
df_pers.show()


+------+-------------+---------+------+-------------------+------------+--------------------+-------------------+--------------------+--------+-------+--------------------+--------------------+
PersNo| EmployeeName|GenderKey|Gender| DOB|SerialNumber| StartDate| EndDate| CreatedDate|IsActive|BatchID| HashAllTS| HashWithStartDate|
+------+-------------+---------+------+-------------------+------------+--------------------+-------------------+--------------------+--------+-------+--------------------+--------------------+
103830| Rey Luke| M| Male|2000-09-21 00:00:00| 1|2021-11-20 07:21:...|9999-12-31 00:00:00|2021-11-20 07:21:...| true| 199|35fd716ca811681b0...|fa68be37188404550...|
103831| Lolly Leades| F|Female|2003-09-01 00:00:00| 2|2021-11-20 07:21:...|9999-12-31 00:00:00|2021-11-20 07:21:...| true| 199|5cb7d5cbfa80c8239...|5453a181707198c73...|
103832| Jordan K| M| Male|1985-10-05 00:00:00| 3|2021-11-20 07:21:...|9999-12-31 00:00:00|2021-11-20 07:21:...| true| 199|e17af8f7d70625978...|9d3acf6ef04322ed4...|
103833|April Lapwood| F|Female|1995-12-11 00:00:00| 4|2021-11-20 07:21:...|9999-12-31 00:00:00|2021-11-20 07:21:...| true| 199|f10cd4488b98acb79...|e3a2264108037d93c...|
103834| Joye Silson| M| Male|1998-01-01 00:00:00| 5|2021-11-20 07:21:...|9999-12-31 00:00:00|2021-11-20 07:21:...| true| 199|dd2af12602db56689...|5c32ff14ae55934a2...|
+------+-------------+---------+------+-------------------+------------+--------------------+-------------------+--------------------+--------+-------+--------------------+--------------------+

In [0]:
jdbcHostname = "testserver19923.database.windows.net"
jdbcDatabase = "testDB"
jdbcPort = 1433
jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbcHostname, jdbcPort, jdbcDatabase)
connectionProperties = {
"user" : "azureuser",
"password" : "Xmb#529Geq12345",
"driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

In [0]:
df_gender = spark.read.jdbc(url=jdbcUrl, table="[dbo].[Gender_M]", properties=connectionProperties)
df_gender.show()

+--------------+----------+------+
GenderMasterID|GenderCode|Gender|
+--------------+----------+------+
 1| M| Male|
 2| F|Female|
+--------------+----------+------+

In [0]:
joined_df = df_pers.join(df_gender, df_pers.GenderKey==df_gender.GenderCode, "left")
#joined_df.show(truncate=False)
joined_df.printSchema()

root
-- PersNo: integer (nullable = true)
-- EmployeeName: string (nullable = true)
-- GenderKey: string (nullable = true)
-- Gender: string (nullable = true)
-- DOB: timestamp (nullable = true)
-- SerialNumber: integer (nullable = true)
-- StartDate: timestamp (nullable = false)
-- EndDate: timestamp (nullable = true)
-- CreatedDate: timestamp (nullable = false)
-- IsActive: boolean (nullable = false)
-- BatchID: integer (nullable = false)
-- HashAllTS: string (nullable = false)
-- HashWithStartDate: string (nullable = false)
-- GenderMasterID: integer (nullable = true)
-- GenderCode: string (nullable = true)
-- Gender: string (nullable = true)

In [0]:
#SELECT ONLY required Columns
req_cols = ["PersNo","EmployeeName","DOB","GenderMasterID","IsActive","StartDate","EndDate","CreatedDate","BatchID"]
final_df = joined_df.select(*req_cols)

In [0]:
final_df.show()

+------+-------------+-------------------+--------------+--------+--------------------+-------------------+--------------------+-------+
PersNo| EmployeeName| DOB|GenderMasterID|IsActive| StartDate| EndDate| CreatedDate|BatchID|
+------+-------------+-------------------+--------------+--------+--------------------+-------------------+--------------------+-------+
103831| Lolly Leades|2003-09-01 00:00:00| 2| true|2021-11-20 07:21:...|9999-12-31 00:00:00|2021-11-20 07:21:...| 199|
103833|April Lapwood|1995-12-11 00:00:00| 2| true|2021-11-20 07:21:...|9999-12-31 00:00:00|2021-11-20 07:21:...| 199|
103830| Rey Luke|2000-09-21 00:00:00| 1| true|2021-11-20 07:21:...|9999-12-31 00:00:00|2021-11-20 07:21:...| 199|
103832| Jordan K|1985-10-05 00:00:00| 1| true|2021-11-20 07:21:...|9999-12-31 00:00:00|2021-11-20 07:21:...| 199|
103834| Joye Silson|1998-01-01 00:00:00| 1| true|2021-11-20 07:21:...|9999-12-31 00:00:00|2021-11-20 07:21:...| 199|
+------+-------------+-------------------+--------------+--------+--------------------+-------------------+--------------------+-------+

In [0]:
final_df.write.jdbc(url=jdbcUrl, table="[dbo].[PersonalData]", mode = "append",properties=connectionProperties)